#Boltzmann Machine

##Downloading the dataset

###ML-100K

In [0]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2020-06-08 15:18:04--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  15.6MB/s    in 0.3s    

2020-06-08 15:18:04 (15.6 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

###ML-1M

In [0]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2020-06-08 15:18:16--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  2.44MB/s    in 2.3s    

2020-06-08 15:18:19 (2.44 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


## Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
from torch.autograd import Variable

## Importing the dataset


In [7]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header = None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [12]:
movies[1:10], movies.shape
#MovieID::Title::Genres

(    0                                   1                             2
 1   2                      Jumanji (1995)  Adventure|Children's|Fantasy
 2   3             Grumpier Old Men (1995)                Comedy|Romance
 3   4            Waiting to Exhale (1995)                  Comedy|Drama
 4   5  Father of the Bride Part II (1995)                        Comedy
 5   6                         Heat (1995)         Action|Crime|Thriller
 6   7                      Sabrina (1995)                Comedy|Romance
 7   8                 Tom and Huck (1995)          Adventure|Children's
 8   9                 Sudden Death (1995)                        Action
 9  10                    GoldenEye (1995)     Action|Adventure|Thriller,
 (3883, 3))

In [13]:
users[1:10], users.shape
#UserID::Gender::Age::Occupation::Zip-code

(    0  1   2   3      4
 1   2  M  56  16  70072
 2   3  M  25  15  55117
 3   4  M  45   7  02460
 4   5  M  25  20  55455
 5   6  F  50   9  55117
 6   7  M  35   1  06810
 7   8  M  25  12  11413
 8   9  M  25  17  61614
 9  10  F  35   1  95370,
 (6040, 5))

In [11]:
ratings[1:10], ratings.shape
#UserID::MovieID::Rating::Timestamp

,0,1,2,3
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


## Preparing the training set and the test set


In [25]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set[1:10],training_set.shape

(   1  1.1  5  874965758
 1  1    3  4  878542960
 2  1    4  3  876893119
 3  1    5  3  889751712
 4  1    7  4  875071561
 5  1    8  1  875072484
 6  1    9  5  878543541
 7  1   11  2  875072262
 8  1   13  5  875071805
 9  1   15  5  875071608,
 (79999, 4))

In [26]:
training_set = np.array(training_set, dtype='int')
training_set
#UserID::MovieID::Rating::Timestamp

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [27]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set[1:10],test_set.shape

(   1   6  5  887431973
 1  1  12  5  878542960
 2  1  14  5  874965706
 3  1  17  3  875073198
 4  1  20  4  887431883
 5  1  23  4  875072895
 6  1  24  3  875071713
 7  1  27  2  876892946
 8  1  31  3  875072144
 9  1  33  4  878542699,
 (19999, 4))

In [28]:
test_set = np.array(test_set, dtype='int')
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

## Getting the number of users and movies


In [29]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

nb_users, nb_movies

(943, 1682)

## Converting the data into an array with users in lines and movies in columns


In [37]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0]==id_users]
        id_ratings = data[:,2][data[:,0]==id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [36]:
print(training_set[1][1:10]) #ratings for each user in list eg user 1, first 10 movie ratings

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0]


In [39]:
print(test_set[1][1:10]) #ratings for each user in list eg user 1, first 10 movie ratings

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## Converting the data into Torch tensors


In [40]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [42]:
training_set.shape, test_set.shape

(torch.Size([943, 1682]), torch.Size([943, 1682]))

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


In [43]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >=3] = 1

In [46]:
training_set[1][1:10]

tensor([-1., -1., -1., -1., -1., -1., -1., -1.,  0.])

In [44]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >=3] = 1

In [47]:
test_set[1][1:10]

tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1.])

## Creating the architecture of the Neural Network


In [70]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh,nv)
        self.a = torch.randn(1,nh)
        self.b = torch.randn(1,nv)
    
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk),0)

In [71]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv,nh)

## Training the RBM


In [72]:
np_epoch = 10

for epoch in range(1, np_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
        s+=1.
    
    print(f"epoch : {epoch} loss : {train_loss/s}")
    

epoch : 1 loss : 0.32029101252555847
epoch : 2 loss : 0.24867312610149384
epoch : 3 loss : 0.24530553817749023
epoch : 4 loss : 0.2451126128435135
epoch : 5 loss : 0.24862882494926453
epoch : 6 loss : 0.24503636360168457
epoch : 7 loss : 0.24537217617034912
epoch : 8 loss : 0.2450951635837555
epoch : 9 loss : 0.2454654723405838
epoch : 10 loss : 0.24695619940757751


## Testing the RBM


In [75]:
test_loss = 0
s = 0.
for id_user in range(0, 1):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0]-v[vt>=0]))
        s+=1.
    
    print(f"test loss : {test_loss/s} ")

test loss : 0.2132352888584137 
